##Cats vs Dogs Class Activation Maps

This time there will only be two classes: Cats and Dogs

In [1]:
##Imports
import tensorflow_datasets as tfds
import tensorflow as tf

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import cv2

#Download and Prep the Dataset

Download via TensorFlow datasets

In [2]:
#Set the new URL because microsoft changed the old one and tensorflow hasn't updated it yet
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")


train_data = tfds.load("cats_vs_dogs", split="train[:80%]", as_supervised=True)
validation_data = tfds.load("cats_vs_dogs", split="train[80%:90%]", as_supervised=True)
test_data = tfds.load("cats_vs_dogs", split="train[-10%:]", as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteOT71YQ/cats_vs_dogs-train.tfrecord


  0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


##Pre-Process the images and create batches before feeding them into the model 

In [3]:
def augment_images(image, label):
  
  #cast to float
  image = tf.cast(image, tf.float32)

  #Normalize the pixel values 
  image = (image / 255)

  #Resize to 300x300
  image = tf.image.resize(image, (300, 300))

  return image, label

In [4]:
#Use the utility function to preprocess the images 
augmented_training_data = train_data.map(augment_images)

#Shuffle and create batches before training 
train_batches = augmented_training_data.shuffle(1024).batch(32)

##Build the Classifier 

The only difference is that the output is just one unit that is sigmoid activated (since there are only two classes)

In [5]:
model = Sequential()

model.add(Conv2D(16, input_shape=(300, 300, 3), kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))

model.add(GlobalAveragePooling2D())

#Output
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 300, 300, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 150, 150, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 37, 37, 64)       0

The loss should be adjusted to deal with just two classes use `binary_cross_entropy`

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])
              
model.fit(train_batches, epochs=25)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/25
582/582 [==============================] - 96s 135ms/step - loss: 0.6609 - accuracy: 0.6032
Epoch 2/25
582/582 [==============================] - 65s 109ms/step - loss: 0.6192 - accuracy: 0.6579
Epoch 3/25
582/582 [==============================] - 65s 109ms/step - loss: 0.5987 - accuracy: 0.6807
Epoch 4/25
582/582 [==============================] - 76s 127ms/step - loss: 0.5811 - accuracy: 0.7018
Epoch 5/25
582/582 [==============================] - 78s 131ms/step - loss: 0.5687 - accuracy: 0.7088
Epoch 6/25
582/582 [==============================] - 78s 129ms/step - loss: 0.5598 - accuracy: 0.7172
Epoch 7/25
582/582 [==============================] - 79s 132ms/step - loss: 0.5493 - accuracy: 0.7231
Epoch 8/25
582/582 [==============================] - 71s 118ms/step - loss: 0.5426 - accuracy: 0.7294
Epoch 9/25
582/582 [==============================] - 81s 135ms/step - loss: 0.5333 - accuracy: 0.7383
Epoch 10/25
582/582 [==============================] - 83s 139ms/step - l

##Building the CAM Model 

Generate the class activation maps 

In [ ]:
gap_weights = model.layers[-1].get_weights()[0]
gap_weights.shape

cam_model = Model(inputs=model.input,
                  outputs=(model.layers[-3].output,
                           model.layers[-1].output)
                  
                  

In [ ]:
def show_cam(image_value, features, results):
  """
  Displays the class activation map of the image 

  Args:
    image_value (tensor) -- preprocessed input image with size (300x300)
    features (array) -- features of the image, shape (1, 37, 37, 128)
    results (array) -- output of the sigmoid layer
  """

  #There is only one image in the batch so it's indexed as `0`
  features_for_img = features[0]
  prediction = results[0]

  #There is only one unit in the output so get the weights connected to it 
  class_activation_weights = gap_weights[:, 0]

  #Upsample to the image size
  class_activation_features = sp.ndimage.zoom(features_for_img, (300/37, 300/37, 1), order=2)

  cam_output = np.dot(class_actiation_features, class_activation weights)

  visualize the result
  print(f'sigmoid output: {results}')
  print(f'prediction: {"dog" if round(results[0][0]) else "cat"}')

  plt.figure(figsize=(8,8))
  plt.imshow(cam_output, cmap='jet', alpha=0.5)
  plt.imshow(tf.squeeze(image_value), alpha=0.5)
  plt.show()

##Testing the Model 

Download some images to see how the class activation maps look like

In [ ]:
!wget -O cat1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat1.jpg
!wget -O cat2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat2.jpg
!wget -O catanddog.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/catanddog.jpg
!wget -O dog1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog1.jpg
!wget -O dog2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog2.jpg

#Utility funtions to preprocess an image and show the CAM

In [ ]:
def convert_and_classify(image):

  #Load the image 
  img = cvg2.imread(image)

  #preprocess the image before feeding it to the model
  img = cv2.resize(img, (300,300)) / 255.0

  #add a batch dimension becasue the model expects it 
  tensor_image = np.expand_dims(img,axis=0)

  #Get the features and prediciton
  features, results = cam_model.predict(tensor_image)

  #Generate the CAM
  show_cam(tensor_image, features, results)

convert_and_classify('cat1.jpg')
convert_and_classify('cat2.jpg')
convert_and_classify('catanddog.jpg')
convert_and_classify('dog1.jpg')
convert_and_classify('dog2.jpg')